In [ ]:
import random
from time import sleep

import pygame

from pinpong.board import *
from pinpong.extension.unihiker import *
from pinpong.board import Board, Pin
from pinpong.extension.unihiker import *

Board().begin()  # Initialize the UNIHIKER


class CarRacing:
    def __init__(self):

        pygame.init()
        self.display_width = 240
        self.display_height = 320
        self.black = (0, 0, 0)
        self.white = (255, 255, 255)
        self.clock = pygame.time.Clock()
        self.gameDisplay = None

        self.initialize()

    def initialize(self):

        self.crashed = False

        self.carImg = pygame.image.load("img/car.png")
        self.carImg = pygame.transform.scale(self.carImg, (40, 60))  # Adjust size as needed
        self.car_x_coordinate = (self.display_width * 0.45)
        self.car_y_coordinate = (self.display_height * 0.8)
        self.car_width = 40

        # enemy_car
        self.enemy_car = pygame.image.load("img/enemy_car_1.png")
        self.enemy_car = pygame.transform.scale(self.enemy_car, (40, 60))  # Adjust size as needed
        self.enemy_car_startx = random.randrange(50, 150)
        self.enemy_car_starty = -60
        self.enemy_car_speed = 3
        self.enemy_car_width = 40
        self.enemy_car_height = 60

        # Background
        self.bgImg = pygame.image.load("img/back_ground.jpg")
        self.bgImg = pygame.transform.scale(self.bgImg, (self.display_width, self.display_height))
        self.bg_x1 = 0
        self.bg_x2 = 0
        self.bg_y1 = 0
        self.bg_y2 = -self.display_height 
        self.bg_speed = 1
        self.count = 0
        
        buzzer.play(buzzer.POWER_UP, buzzer.Once)  # Play music once
        buzzer.play(buzzer.FUNK, buzzer.ForeverInBackground)  # Play music continuously in the background

    def car(self, car_x_coordinate, car_y_coordinate):
        self.gameDisplay.blit(self.carImg, (car_x_coordinate, car_y_coordinate))

    def racing_window(self):
        self.gameDisplay = pygame.display.set_mode((self.display_width, self.display_height))
        pygame.display.set_caption('Car Dodge')
        self.run_car()

    def run_car(self):

        while not self.crashed:

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.crashed = True

                if (event.type == pygame.KEYDOWN):
                    if (event.key == pygame.K_LEFT):
                        self.car_x_coordinate -= 10
                    if (event.key == pygame.K_RIGHT):
                        self.car_x_coordinate += 10
            
            
            if(accelerometer.get_y() > 0.05):
                self.car_x_coordinate -= 10
                if(self.car_x_coordinate < 50):
                    self.car_x_coordinate = 50
            if(accelerometer.get_y() < -0.05):
                self.car_x_coordinate += 10
                if(self.car_x_coordinate > 150):
                    self.car_x_coordinate = 150

            self.gameDisplay.fill(self.black)
            self.back_ground_raod()

            self.run_enemy_car(self.enemy_car_startx, self.enemy_car_starty)
            self.enemy_car_starty += self.enemy_car_speed

            if self.enemy_car_starty > self.display_height:
                self.enemy_car_starty = 0 - self.enemy_car_height
                self.enemy_car_startx = random.randrange(50, 150)

            self.car(self.car_x_coordinate, self.car_y_coordinate)
            self.highscore(self.count)
            self.count += 1
            if (self.count % 100 == 0):
                self.enemy_car_speed += 1
                self.bg_speed += 1
            if self.car_y_coordinate < self.enemy_car_starty + self.enemy_car_height:
                if self.car_x_coordinate > self.enemy_car_startx and self.car_x_coordinate < self.enemy_car_startx + self.enemy_car_width or self.car_x_coordinate + self.car_width > self.enemy_car_startx and self.car_x_coordinate + self.car_width < self.enemy_car_startx + self.enemy_car_width:
                    buzzer.stop()  # Stop playing in the background
                    buzzer.play(buzzer.POWER_DOWN, buzzer.Once)  # Play music once
                    self.crashed = True
                    self.display_message("Game Over !!!")

            if self.car_x_coordinate < 0 or self.car_x_coordinate > self.display_width - self.car_width:
                buzzer.stop()  # Stop playing in the background
                buzzer.play(buzzer.POWER_DOWN, buzzer.Once)  # Play music once
                self.crashed = True
                self.display_message("Game Over !!!")

            pygame.display.update()
            self.clock.tick(30)

    def display_message(self, msg):
        font = pygame.font.SysFont("comicsansms", 36, True)
        text = font.render(msg, True, (255, 255, 255))
        self.gameDisplay.blit(text, (self.display_width // 2 - text.get_width() // 2, self.display_height // 2 - text.get_height() // 2))
        self.display_credit()
        pygame.display.update()
        self.clock.tick(60)
        sleep(1)
        car_racing.initialize()
        car_racing.racing_window()

    def back_ground_raod(self):
        self.gameDisplay.blit(self.bgImg, (self.bg_x1, self.bg_y1))
        self.gameDisplay.blit(self.bgImg, (self.bg_x2, self.bg_y2))

        self.bg_y1 += self.bg_speed
        self.bg_y2 += self.bg_speed

        if self.bg_y1 >= self.display_height:
            self.bg_y1 = -self.display_height

        if self.bg_y2 >= self.display_height:
            self.bg_y2 = -self.display_height

    def run_enemy_car(self, thingx, thingy):
        self.gameDisplay.blit(self.enemy_car, (thingx, thingy))

    def highscore(self, count):
        font = pygame.font.SysFont("arial", 16)
        text = font.render("Score : " + str(count), True, self.white)
        self.gameDisplay.blit(text, (0, 0))

    def display_credit(self):
        font = pygame.font.SysFont("lucidaconsole", 10)
        text = font.render("Thanks for playing!", True, self.white)
        self.gameDisplay.blit(text, (self.display_width - text.get_width() - 10, self.display_height - text.get_height() - 10))


if __name__ == '__main__':
    car_racing = CarRacing()
    car_racing.racing_window()
